In [1]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

In [2]:
p1 = pd.read_csv('201805012_7all_2050.txt', sep=' ')
p2 = pd.read_csv('20180513-2900-day-47.txt', sep=' ')
p3 = pd.read_csv('20180514-2200-day-4567.txt', sep=' ')


p2['predicted_score'] = p1['predicted_score'] * 0.4 + p2['predicted_score'] * 0.3 + p3['predicted_score'] * 0.3


p2[['instance_id', 'predicted_score']].to_csv(
    '20180514-combination.txt', index=False, sep=' ')

test_ctr = float(sum(p2['predicted_score'])/float(len(p2['predicted_score'])))
test_ctr

0.03919176900567571

In [3]:
all_data_path = feature_data_path + 'all_data_all_features_new_0512.pkl'
all_data = load_pickle(all_data_path)
data = all_data[all_data.day == 7]

In [4]:
predict = pd.read_csv('20180514-combination.txt', sep=' ')

del predict['instance_id']
test_data = data[data.is_trade == -2]

test_data = test_data.reset_index()
predict = predict.reset_index()

test_data = pd.concat([test_data, predict], axis=1)

ctr = test_data.groupby(['hour'])['predicted_score'].mean().reset_index().rename(columns={0: 'ctr'})
ctr

,hour,predicted_score
0,12,0.042264
1,13,0.041634
2,14,0.040280
3,15,0.040590
4,16,0.041031
5,17,0.041727
6,18,0.041745
7,19,0.041559
8,20,0.039402
9,21,0.036455
